# Finite Difference Stencils

We want to represent $f'(0)$ in the following form (say up to $O(h^5)$):
$$
f'(0) = \frac{c_1 f(-2h) + c_2 f(-h) + c_3 f(+h) + c_4 f(+2h)}{h}
$$
which should be ture for any analytic function, for the first few bases, we can choose:
$$
\begin{align}
f(x) = 1, f(x) = x, f(x) = x^2, f(x) = x^3.
\end{align}
$$
Take derivatives
$$
\begin{align}
\left.
\begin{array}{c}
\frac{c_1 + c_2 + c_3 + c_4}{h} &= 0,\\
\frac{c_1 (-2h) + c_2 (-h) + c_3 (h) + c_4 (h)}{h} &= 1,\\
\frac{c_1 (-2h)^2 + c_2 (-h)^2 + c_3 (h)^2 + c_4 (2h)^2}{h} &= 0,\\
\frac{c_1 (-2h)^3 + c_2 (-h)^3 + c_3 (h)^3 + c_4 (2h)^3}{h} &= 0,\\
\end{array}
\right\}
%\begin{array}{c}
%
%\end{array}
\end{align}
$$

In [1]:
include("stencils.jl")

Main.FDStencils

In [2]:
using .FDStencils

In [3]:
simple = [-3//2, -1//2, 1//2, 3//2];
values = [0, 1, 0, 0];

In [5]:
cs = GetCoefficient(simple, values)

4-element Vector{Rational{Int64}}:
  1//24
 -9//8
  9//8
 -1//24

## Write to Files

In [35]:
filename = "test.hxx";
if isfile(filename)
    println("file ", filename, " already, exist, replacing it ...")
    rm(filename)
end
filepointer = open("test.hxx", "a")
pr(x="") = println(filepointer, x)

file test.hxx already, exist, replacing it ...


pr (generic function with 2 methods)

In [36]:
pr("#ifndef FD_HXX")
pr("#define FD_HXX")
pr()
pr("#include <cctk.h>")
pr("namespace WaveToyHigherOrderX {")

In [37]:
open("test.hxx", "a") do file
    println(file, cs[1])
    println(file, cs[2])
end

In [38]:
pr("} // namespace WaveToyHigherOrderX")
pr("#endif // #ifndef FD_HXX")

In [39]:
close(filepointer)